In [287]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import shutil

In [288]:
# Xavier Initialization
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [289]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [290]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [291]:
# Generator Net
Z = tf.placeholder(tf.float32, shape=[None, 100], name='Z')
G_W1 = tf.Variable(xavier_init([100, 128]), name='G_W1')
G_b1 = tf.Variable(tf.zeros(shape=[128]), name='G_b1')
G_W2 = tf.Variable(xavier_init([128, 784]), name='G_W2')
G_b2 = tf.Variable(tf.zeros(shape=[784]), name='G_b2')

theta_G = [G_W1, G_W2, G_b1, G_b2]

def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

In [292]:
# Discriminator Net
X = tf.placeholder(tf.float32, shape=[None, 784], name='X')
D_W1 = tf.Variable(xavier_init([784, 128]), name='D_W1')
D_b1 = tf.Variable(tf.zeros(shape=[128]), name='D_b1')
D_W2 = tf.Variable(xavier_init([128, 1]), name='D_W2')
D_b2 = tf.Variable(tf.zeros(shape=[1]), name='D_b2')

theta_D = [D_W1, D_W2, D_b1, D_b2]

def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_h1_dropout = tf.nn.dropout(D_h1, 0.8)
    D_logit = tf.matmul(D_h1_dropout, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

In [293]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

# D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
# G_loss = -tf.reduce_mean(tf.log(D_fake))

D_loss_real = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        labels=tf.ones_like(D_logit_real),
        logits=D_logit_real
    )
)
D_loss_fake = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        labels=tf.zeros_like(D_logit_fake),
        logits=D_logit_fake
    )
)
D_loss = D_loss_real + D_loss_fake
D_loss_sum = tf.summary.scalar('D_loss', D_loss)
G_loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        labels=tf.ones_like(D_logit_fake),
        logits=D_logit_fake
    )
)
G_loss_sum = tf.summary.scalar('G_loss', G_loss)
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

mb_size = 128
Z_dim = 100

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

sess = tf.Session()
writer = tf.summary.FileWriter('result', sess.graph)
sess.run(tf.global_variables_initializer())

if os.path.exists('out/'):
    shutil.rmtree('out/')
os.makedirs('out/')

i = 0
for it in range(100000):
    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})
        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)
    _, D_loss_curr, s1 = sess.run([D_solver, D_loss, D_loss_sum], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr, s2 = sess.run([G_solver, G_loss, G_loss_sum], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        writer.add_summary(s1, it)
        writer.add_summary(s2, it)
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter: 0
D loss: 1.079
G_loss: 3.005

Iter: 1000
D loss: 0.04477
G_loss: 5.105

Iter: 2000
D loss: 0.2059
G_loss: 3.99

Iter: 3000
D loss: 0.1941
G_loss: 3.446

Iter: 4000
D loss: 0.3815
G_loss: 3.111

Iter: 5000
D loss: 0.2786
G_loss: 2.77

Iter: 6000
D loss: 0.4854
G_loss: 2.645

Iter: 7000
D loss: 0.5194
G_loss: 2.056

Iter: 8000
D loss: 0.6008
G_loss: 2.305

Iter: 9000
D loss: 0.6885
G_loss: 1.876

Iter: 10000
D loss: 0.6687
G_loss: 2.053

Iter: 11000
D loss: 0.8273
G_loss: 1.765

Iter: 12000
D loss: 0.6994
G_loss: 2.165

Iter: 13000
D loss: 0.693
G_loss: 2.053

Iter: 14000
D loss: 0.8414
G_loss: 2.089

Iter: 15000
D loss: 0.6876
G_loss: 2.024

Iter: 16000
D loss: 0.7709
G_loss: 1.731

Iter: 17000
D loss: 0.8485
G_loss: 1.78

Iter: 18000
D loss: 0.7778
G_loss: 1.678

Iter: 19000
D loss: 0.794